# 실시간 Lip Landmark 추출

---

```실험 전```
- 사람&문장&반복횟수 **초기화**
----

```실험 동안```
- 코드 실행
- 코드에서 lip localization 한 값들이 뜨기 시작하면 시작
- 입을 벌리면 Frame 별 저장 시작
- 최대한 크게 최대한 가까이 최대한 밝은 공간에서 할 것!
- 말은 조금은 느리게 (3초에 한단어씩이란 생각으로)
- 웹캠 끌때는 실행되고 있는 웹캠에서 'q' 눌러서 끄기

---

```실험후```
- 웹캠 반응이 없는데 계속 돌아가고 있다면, 위에 stop 눌러서 꺼주기
- 가끔 웹캠 반응 없으면 웹캠 종료 버튼 
- 저장되어있는거 확인
- 앞이랑 뒤에 상관없는 사진들도 저장이 되었다면 삭제(입을 벌리기 직전 다물기 직전까지만 잘라서 저장: 수작업)
- mouth 사진 저장되어있는거 앞뒤로 확인해서 mouth가 검출이 안되었는데 저장된 것이 있다면 **그 문장은 다시 찍기**
- 결과 값 csv 파일로 저장
-  **커널 다시 돌리기**

In [1]:
import pandas as pd
import dlib
import cv2
import numpy as np
import os

In [2]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [3]:
def get_rect(shape):
    rw = 0
    rh = 0
    rx = 65535
    ry = 65535
    for (x,y) in shape:
        rw = max(rw,x)
        rh = max(rh,y)
        rx = min(rx,x)
        ry = min(ry,y)
    return (rx,ry,rw-rx,rh-ry)

In [4]:
######################################
# person & sentences
person = 'F01' 
sentence= '01'
num = '00'
#######################################
count = 0
dataset = {}
dataset[person] = {}
dataset[person][sentence] ={}
dataset[person][sentence][num] ={}

# create face detector, predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat") # predict 해주는애

# web cam start
video_capture = cv2.VideoCapture(0) 

while True:
    ret, frame = video_capture.read() #Get Frame: ret True, no: False

   # resize the video
    image = cv2.resize(frame, dsize=(640, 480), interpolation=cv2.INTER_AREA)
    img = image.copy()
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Get faces (up-sampling=1)
    face_detector = detector(img_gray, 1)
    
    if len(face_detector) == 0:
        print("No face Detected,%d", int(video_capture.get(1)))
    
    # for loop for faces
    for face in face_detector:
        #cv2.rectangle(image, (face.left(), face.top()), (face.right(), face.bottom()),(0, 0, 255), 3)

        # make prediction and transform to numpy array
        landmarks = predictor(image, face)  # 얼굴에서 68개 점 찾기
        landmarks = shape_to_np(landmarks)
        lip_landmarks = [landmarks[x] for x in range(48, 68)]
        
        for (i, (x, y)) in enumerate(lip_landmarks):
            cv2.circle(image, (x, y), 1, (255, 255, 0), -1)
        
        (x, y, w, h) = get_rect(lip_landmarks)

        for s in lip_landmarks:
            s[0] -= x
            s[1] -= y
        

    cv2.imshow('result', image)
    
    # exit camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    dataset[person][sentence][num][count] ={}   
    if (int(video_capture.get(1)) % 3 == 0): # after 5 sec passed (5초 단위)
        path = 'cutimgs/%s/%s/%s'%(person, sentence, num)
        
        #path 없다면 만들고 있다면 pass
        try:
            os.makedirs(path)
        except FileExistsError as e:
            pass
        img_name = f'img_{count}.jpg'
        mouth_name = f'mouth_{count}.jpg'
        img_path = path + '/' + img_name
        mouth_path = path + '/' + mouth_name
        # save images by frame
        cv2.imwrite(img_path, img)
        cv2.imwrite(mouth_path, image) # 확인용
        print(lip_landmarks,"\n")
        print('Saved frame number : ' + str(int(video_capture.get(1))))
        dataset[person][sentence][num][count]= lip_landmarks  
        
        count += 1
        
        
        # exit camera
    if cv2.waitKey(1) == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

[array([ 0, 17]), array([7, 7]), array([16,  1]), array([23,  2]), array([30,  0]), array([40,  4]), array([50, 12]), array([41, 21]), array([33, 25]), array([25, 27]), array([17, 26]), array([ 9, 24]), array([ 5, 16]), array([16, 12]), array([24, 12]), array([31, 11]), array([45, 12]), array([32, 12]), array([25, 13]), array([17, 13])] 

Saved frame number : 1213152
[array([ 0, 18]), array([7, 8]), array([16,  2]), array([24,  3]), array([31,  0]), array([41,  4]), array([51, 12]), array([43, 21]), array([35, 26]), array([26, 28]), array([18, 28]), array([ 9, 25]), array([ 5, 16]), array([16, 12]), array([24, 12]), array([32, 11]), array([46, 12]), array([33, 13]), array([25, 15]), array([17, 14])] 

Saved frame number : 1213164
[array([ 0, 17]), array([8, 7]), array([16,  1]), array([23,  3]), array([30,  0]), array([40,  5]), array([51, 12]), array([41, 21]), array([33, 25]), array([25, 27]), array([18, 26]), array([ 9, 23]), array([ 6, 16]), array([16, 12]), array([24, 12]), array(

In [7]:
#dataset

In [6]:
fileCount = 0
shapes = []
for count in dataset[person][sentence][num].keys():
    shape = dataset[person][sentence][num][count]
    for (x,y) in shape:
        shapes.append((num,count,x,y))

df = pd.DataFrame(data = shapes,columns = ['num','frame','x','y'])
path = "cutimgs/%s/%s" % (person,sentence)
try:
    os.makedirs(path)
except FileExistsError as e:
    pass
filename = "shapes_%s.csv" %(num)
csvPath = "%s/%s" % (path,filename)
print("saving %s" % csvPath)
df.to_csv(csvPath)

saving cutimgs/F01/01/shapes_00.csv
